In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from kenpompy.utils import login
import numpy as np

In [63]:
convert_dict = {'MIN':int,
               'OREB':int,
               'DREB':int,
               'REB':int,
               'AST':int,
               'STL':int,
               'BLK':int,
               'TO':int,
               'PF':int,
               'PTS':int,
               'FGM':int,
               'FGA':int,
               '3PM':int,
               '3PA':int,
               'FTM':int,
               'FTA':int}
col_order = ['Player',
                 'PTS',
                 'MIN',
                 'FGM',
                 'FGA',
                 '3PM',
                 '3PA',
                 'FTM',
                 'FTA',
                 'OREB',
                 'DREB',
                 'REB',
                 'AST',
                 'STL',
                 'BLK',
                 'TO',
                 'PF',
                 'Position']

def calculate_possessions(fga,orebs,tos,fta):
    value = (fga-orebs) + tos + (0.475*fta)
    return value

def clean_up_boxscore(df):
    df = df.iloc[:len(df)-2,]
    df['Position'] = [i[-1] for i in df.Starters]
    df = df.infer_objects()
    df.Starters = [i[:int((len(i)-1)/2)] for i in df.Starters]
    df['FGM'] = [i[0] for i in df.FG.str.split('-')]
    df['FGA'] = [i[1] for i in df.FG.str.split('-')]
    df['3PM'] = [i[0] for i in df['3PT'].str.split('-')]
    df['3PA'] = [i[1] for i in df['3PT'].str.split('-')]
    df['FTM'] = [i[0] for i in df.FT.str.split('-')]
    df['FTA'] = [i[1] for i in df.FT.str.split('-')]
    df = df.astype(convert_dict)
    df = df.append(df.sum(numeric_only = True),ignore_index = True)
    last_row = len(df) - 1
    df = df.rename(columns = {"Starters":"Player"})
    df = df[col_order]
    df.iloc[last_row,0] = 'Team'
    df.iloc[last_row,17] = ""
    df = df.fillna(0)
    return df.astype(convert_dict)

In [23]:
game_id = 401364412
url = 'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/'+str(game_id)
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
print(str(soup.find('title')).split('-')[0].replace("<title>","").strip())
print(str(soup.find("title")).split("-")[2].strip())

Ohio State vs. Purdue
January 30, 2022


AWAY TEAM IS ALWAYS FIRST

In [24]:
away_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[0]
home_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[1]
game_date = str(soup.find("title")).split("-")[2].strip()
print(f"{away_team} at {home_team}")

Ohio State at Purdue


Away team is always index value 1

In [66]:
dfs = pd.read_html(url)
away_team_df = dfs[1]
home_team_df = dfs[2]
if type(away_team_df.columns[0]) is tuple:
  away_team_df.columns = [i[0] for i in away_team_df.columns.tolist()]
if type(home_team_df.columns[0]) is tuple:
  home_team_df.columns = [i[0] for i in home_team_df.columns.tolist()]

away_team_df = clean_up_boxscore(away_team_df)
home_team_df = clean_up_boxscore(home_team_df)
display(away_team_df,home_team_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Player,PTS,MIN,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TO,PF,Position
0,E.J. Liddell,20,36,7,16,2,7,4,5,1,2,3,3,1,2,0,4,F
1,Z. Key,1,12,0,0,0,0,1,2,1,0,1,0,0,0,1,4,F
2,M. Johnson Jr.,3,21,1,5,1,2,0,0,1,2,3,1,1,0,2,1,G
3,M. Branham,20,31,7,10,2,3,4,5,0,7,7,1,1,0,0,3,G
4,J. Wheeler,2,24,1,3,0,1,0,0,0,3,3,2,2,0,2,1,G
5,J. Ahrens,3,11,1,4,1,4,0,0,0,0,0,1,0,0,1,0,F
6,K. Young,7,19,3,10,1,4,0,0,2,0,2,2,0,0,0,3,F
7,J. Brunk,4,13,2,2,0,0,0,0,2,1,3,1,0,2,0,3,C
8,E. Brown III,9,20,3,5,1,3,2,2,2,1,3,2,0,0,0,2,G
9,C. Russell,9,13,4,8,1,5,0,0,0,2,2,2,1,0,0,1,G


,Player,PTS,MIN,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TO,PF,Position
0,M. Gillis,7,28,2,5,1,3,2,2,5,6,11,2,1,0,2,1,F
1,Z. Edey,20,21,7,9,0,0,6,11,3,0,3,0,0,1,2,3,C
2,I. Thompson,0,18,0,3,0,1,0,0,0,1,1,0,0,0,0,0,G
3,E. Hunter Jr.,10,31,4,7,1,1,1,2,1,2,3,5,1,0,2,1,G
4,S. Stefanovic,11,31,3,7,2,5,3,3,0,3,3,4,0,0,1,1,G
5,C. Furst,0,12,0,0,0,0,0,0,0,1,1,1,0,0,0,4,F
6,T. Williams,9,19,4,8,0,0,1,5,2,6,8,3,2,2,1,1,F
7,J. Ivey,21,31,8,20,3,6,2,3,1,2,3,2,0,1,2,1,G
8,E. Morton,3,9,1,1,1,1,0,0,0,0,0,0,0,0,0,0,G
9,Team,81,200,29,60,8,17,15,26,12,21,33,17,4,4,10,12,


In [73]:
dfs[1]

,Starters,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS
,Bench,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS
0,E.J. LiddellE.J. LiddellF,36.0,7-16,2-7,4-5,1.0,2.0,3.0,3.0,1.0,2.0,0.0,4.0,20.0
1,Z. KeyZ. KeyF,12.0,0-0,0-0,1-2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0
2,M. Johnson Jr.M. Johnson Jr.G,21.0,1-5,1-2,0-0,1.0,2.0,3.0,1.0,1.0,0.0,2.0,1.0,3.0
3,M. BranhamM. BranhamG,31.0,7-10,2-3,4-5,0.0,7.0,7.0,1.0,1.0,0.0,0.0,3.0,20.0
4,J. WheelerJ. WheelerG,24.0,1-3,0-1,0-0,0.0,3.0,3.0,2.0,2.0,0.0,2.0,1.0,2.0
5,J. AhrensJ. AhrensF,11.0,1-4,1-4,0-0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0
6,K. YoungK. YoungF,19.0,3-10,1-4,0-0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,3.0,7.0
7,J. BrunkJ. BrunkC,13.0,2-2,0-0,0-0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,3.0,4.0
8,E. Brown IIIE. Brown IIIG,20.0,3-5,1-3,2-2,2.0,1.0,3.0,2.0,0.0,0.0,0.0,2.0,9.0
